In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Data preprocessing

In [ ]:
test_dataset = np.load('').T # Input the path to your test dataset here
labels = np.load('') # Input the path to your test labels here

In [ ]:
data_test_size = test_dataset.shape[0]
data_test_shaped = np.zeros(shape=(data_test_size,300,300,3))

for image in range(data_test_size):
    one_img = test_dataset[image].reshape((300, 300, 3))
    data_test_shaped[image] = one_img
data_test_shaped.shape

In [ ]:
import gc
del test_dataset
gc.collect()

In [ ]:
data_test_shaped = data_test_shaped.astype('uint8')
labels_test = labels.astype('uint8')

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((data_test_shaped, labels))

In [ ]:
BATCH_SIZE = 42
SHUFFLE_BUFFER_SIZE = 100

test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset

Show the first nine images and labels from the training set:

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in test_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(labels[i].numpy())
    plt.axis("off")

As the original dataset doesn't contain a test set, you will create one. To do so, determine how many batches of data are available in the validation set using `tf.data.experimental.cardinality`, then move 20% of them to a test set.

In [ ]:
print('Number of train batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [ ]:
for image_batch, labels_batch in test_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

test_ds = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
Nocturna = tf.keras.models.load_model('NocturnaEmblemClassifierV3.h5')

In [ ]:
predictions = Nocturna.predict(test_ds)

In [ ]:
class_names = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
y_test = np.zeros(shape=predictions.shape[0])
for i in range(predictions.shape[0]):
    score = tf.nn.softmax(predictions[i])
    y_test[i] = class_names[np.argmax(score)]
    

In [ ]:
test_cm = confusion_matrix(labels_test, y_test)

In [ ]:
ConfusionMatrixDisplay(val_cm).plot()

In [ ]:
misclassified = 0
for i in range(labels_test.shape[0]):
    if y_test[i] != labels_test[i]:
      misclassified += 1
misclassified

In [ ]:
classification = np.array(list(map(lambda x, y: x == y, y_test, labels_test)))
np.unique(classification, return_counts=True)

In [ ]:
j = 0
k = 0
plt.figure(figsize=(30, 500))
for images, labels in test_ds.take(tf.data.experimental.cardinality(test_dataset)):
    for i in range(images.shape[0]):
      if not classification[k]:
        ax = plt.subplot(50, 4, j + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f'{y_test[k]}, {labels[i]}')
        plt.axis("off")
        j += 1
      k += 1
print(j)

In [ ]:
print(classification_report(labels_test, y_test))